In [ ]:
!pip install pylatexenc

In [ ]:
!pip install qiskit

In [ ]:
!pip install qiskit-ibm-runtime

In [ ]:
!pip install qiskit-aer

In [ ]:
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

# Create empty circuit
example_circuit = QuantumCircuit(2) # Create a Quantum Circuit with 2 registers (qubits) with both state 0
example_circuit.measure_all() # Adds measurement to all qubits.
example_circuit.draw("mpl").savefig("test.svg")
# example_circuit.draw("mpl", filename='./name.svg')

In [ ]:
!pwd

In [ ]:
# You'll need to specify the credentials when initializing QiskitRuntimeService, if they were not previously saved.
service = QiskitRuntimeService()
backend = service.backend("ibmq_qasm_simulator")
job = Sampler(backend).run(example_circuit)
print(f"job id: {job.job_id()}")
result = job.result()
print(result)

In [ ]:
from qiskit import QuantumCircuit
from pylatexenc import *

# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
qc = QuantumCircuit(2)

# Add a Hadamard gate to qubit 0
qc.h(0)

# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)

# Return a drawing of the circuit using MatPlotLib ("mpl"). This is the
# last line of the cell, so the drawing appears in the cell output.
# Remove the "mpl" argument to get a text drawing.

qc.draw("mpl")

In [ ]:
from qiskit.quantum_info import Pauli

ZZ = Pauli('ZZ') # create the two-qubit Pauli operator: Z on qubit 1 and Z on qubit 2.
ZI = Pauli('ZI')
IZ = Pauli('IZ')
XX = Pauli('XX')
XI = Pauli('XI')
IX = Pauli('IX')
# If the state is entangled, then the correlation between qubit 1 and qubit 2 is one.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options

service = QiskitRuntimeService()

# Run on the least-busy backend you have access to
# backend = service.least_busy(simulator=False, operational=True)
backend = service.backend("ibmq_qasm_simulator")

options = Options()
options.resilience_level = 1 # resilience_level 1: Mitigate error associated with readout errors
options.optimization_level = 3 # Higher levels generate more optimized circuits, at the expense of longer compile times.

# Create an Estimator object
estimator = Estimator(backend, options=options) # service estimating expectation values of quantum circuits and observables.

# Submit the circuit to Estimator
job = estimator.run(circuits=[qc]*6, observables=[IZ, IX, ZI, XI, ZZ, XX], shots = 5000)

# Once the job is complete, get the result
job.result()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# data
data = ['IZ', 'IX', 'ZI', 'XI', 'ZZ', 'XX']
values = job.result().values

# creating error bars
error = []
for case in job.result().metadata:
    error.append(2*np.sqrt(case['variance']/case['shots']))

# plotting graph
plt.plot(data, values)
plt.errorbar(data, values, yerr = error, fmt ='o')
plt.xlabel('Observables')
plt.ylabel('Values')
plt.show()

### repetition codes

In [ ]:
# Qiskit imports
from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.result import marginal_counts

# Qiskit IBM Provider
from qiskit_ibm_provider import IBMProvider, least_busy

import warnings
warnings.filterwarnings("ignore")

In [ ]:
provider = IBMProvider() # Provides access to the IBM Quantum services
# backend = least_busy(provider.backends(simulator=False, operational=True))
backend = service.backend("ibmq_qasm_simulator")
# backend = service.backend("ibm_brisbane")
# backend = service.backend("ibm_osaka")
backend

In [ ]:
qreg_measure = QuantumRegister(2, name="syn_q")
qreg_data = QuantumRegister(3, name="q")
creg_syndrome = ClassicalRegister(2, name="syn_c")
creg_data = ClassicalRegister(3, name="c")
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]

def build_qc():
    """Build a typical error correction circuit"""
    return QuantumCircuit(qreg_measure, qreg_data, creg_syndrome, creg_data)

def initialize_qubits(circuit: QuantumCircuit):
    """Initialize qubit to |1>"""
    circuit.x(qreg_data[0])
    # circuit.barrier(qreg_data) # A barrier is a visual indicator of the grouping of a circuit section.
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def encode_bit_flip(circuit, state, ancillas) -> QuantumCircuit: # -> is a return value annotation
    """Encode bit-flip. This is done by simply adding a cx"""
    for ancilla in ancillas:
        circuit.cx(state, ancilla)
    # circuit.barrier(state, *ancillas)
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_measure):
    """
        Measure the syndrome by measuring the parity.
        We reset our ancilla qubits after measuring the stabilizer
        so we can reuse them for repeated stabilizer measurements.
        Because we have already observed the state of the qubit,
        we can write the conditional reset protocol directly to
        avoid another round of qubit measurement if we used
        the `reset` instruction.
    """
    circuit.barrier(*qreg_data, *qreg_measure)
    circuit.cx(qreg_data[0], qreg_measure[0])
    circuit.cx(qreg_data[1], qreg_measure[0])
    circuit.cx(qreg_data[0], qreg_measure[1])
    circuit.cx(qreg_data[2], qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure) # * can pass arbitrary number argument
    circuit.measure(qreg_measure, creg_measure)
    # with circuit.if_test((creg_syndrome[0], 1)): # reset the ancilla bit
    #     circuit.x(qreg_measure[0])
    # with circuit.if_test((creg_syndrome[1], 1)):
    #     circuit.x(qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def apply_correction_bit(circuit, qreg_data, creg_syndrome):
    """We can detect where an error occurred and correct our state"""
    circuit.x(qreg_data[0]).c_if(creg_syndrome, 3)
    circuit.x(qreg_data[1]).c_if(creg_syndrome, 1)
    circuit.x(qreg_data[2]).c_if(creg_syndrome, 2)
    # with circuit.if_test((creg_syndrome, 3)):
    #     circuit.x(qreg_data[0]).c_if(creg_syndrome, 3)
    # with circuit.if_test((creg_syndrome, 1)):
    #     circuit.x(qreg_data[1])
    # with circuit.if_test((creg_syndrome, 2)):
    #     circuit.x(qreg_data[2])
    # circuit.barrier(qreg_data)
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def apply_final_readout(circuit, qreg_data, creg_data):
    """Read out the final measurements"""
    # circuit.barrier(qreg_data)
    circuit.measure(qreg_data, creg_data)
    return circuit

In [ ]:
def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit.x(qreg_data[0])
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=True)
# circuit.draw(output="mpl", style='iqp')
# circuit.draw(output="mpl").savefig("Circuit: Bit Flip Code.svg")
circuit.draw(output="mpl")

In [ ]:
from qiskit.circuit import IfElseOp

if "if_else" not in backend.target:
    backend.target.add_instruction(IfElseOp, name="if_else")

In [ ]:
transpiled_circuit = transpile(circuit, backend) # select the best qubit layout and routing # basis gate set
# transpiled_circuit = circuit
transpiled_circuit.draw(output="mpl", style='iqp', idle_wires=False)

In [ ]:
no_correction_circuit = build_error_correction_sequence(apply_correction=False)
transpiled_no_correction_circuit = transpile(no_correction_circuit, backend)
# transpiled_no_correction_circuit = no_correction_circuit

In [ ]:
job_with_correction = backend.run(transpiled_circuit, shots=500, dynamic=True)
result_with_correction = job_with_correction.result()

In [ ]:
job_no_correction = backend.run(transpiled_no_correction_circuit, shots=500, dynamic=True)
result_no_correction = job_no_correction.result()

In [ ]:
data_indices = list(range(len(qreg_data)))
syndrome_indices = list(range(data_indices[-1]+1, len(qreg_data) + len(qreg_measure) ))

def decode_result(data_counts, syndrome_counts):
    shots = sum(data_counts.values())
    success_trials = data_counts.get('000', 0) + data_counts.get('111', 0)
    failed_trials = shots-success_trials
    error_correction_events = shots-syndrome_counts.get('00', 0)
    print(f"Bit flip errors were detected/corrected on {error_correction_events}/{shots} trials")
    print(f"A final parity error was detected on {failed_trials}/{shots} trials")

In [ ]:
non_corrected_marginalized_data_result = marginal_counts(result_no_correction, data_indices)
non_corrected_marginalized_syndrome_result = marginal_counts(result_no_correction, syndrome_indices)

print(f'Completed bit code experiment data measurement counts {non_corrected_marginalized_data_result.get_counts(0)}')
print(f'Completed bit code experiment syndrome measurement counts {non_corrected_marginalized_syndrome_result.get_counts(0)}')
decode_result(non_corrected_marginalized_data_result.get_counts(0), non_corrected_marginalized_syndrome_result.get_counts(0))

In [ ]:
corrected_marginalized_data_result = marginal_counts(result_with_correction, data_indices)
corrected_marginalized_syndrome_result = marginal_counts(result_with_correction, syndrome_indices)

print(f'Completed bit code experiment data measurement counts {corrected_marginalized_data_result.get_counts(0)}')
print(f'Completed bit code experiment syndrome measurement counts {corrected_marginalized_syndrome_result.get_counts(0)}')
decode_result(corrected_marginalized_data_result.get_counts(0), corrected_marginalized_syndrome_result.get_counts(0))

### Noise Simulation

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError, pauli_error, depolarizing_error, thermal_relaxation_error)

In [ ]:
import qiskit
from qiskit_aer import AerSimulator

# Generate 3-qubit GHZ state
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure_all()

# Construct an ideal simulator
aersim = AerSimulator()

# Perform an ideal simulation
result_ideal = qiskit.execute(circ, aersim).result()
counts_ideal = result_ideal.get_counts(0)
print('Counts(ideal):', counts_ideal)
# Counts(ideal): {'000': 493, '111': 531}

In [ ]:
# circ.draw(output="mpl", style='iqp')
circ.draw("mpl")

In [ ]:
from qiskit import IBMQ, transpile
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

In [ ]:
from qiskit.providers.fake_provider import FakeVigo
device_backend = FakeVigo()

In [ ]:
# Construct quantum circuit
circ = QuantumCircuit(3, 3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure([0, 1, 2], [0, 1, 2])

sim_ideal = AerSimulator()

# Execute and get counts
result = sim_ideal.run(transpile(circ, sim_ideal)).result()
counts = result.get_counts(0)
plot_histogram(counts, title='Ideal counts for 3-qubit GHZ state')

In [ ]:
sim_vigo = AerSimulator.from_backend(device_backend)

In [ ]:
# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

### Repetition code on Aer

In [ ]:
qreg_data = QuantumRegister(3)
qreg_measure = QuantumRegister(2)
creg_data = ClassicalRegister(3)
creg_syndrome = ClassicalRegister(2)
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]

def build_qc():
    """Build a typical error correction circuit"""
    return QuantumCircuit(qreg_data, qreg_measure, creg_data, creg_syndrome)

def initialize_qubits(circuit: QuantumCircuit):
    """Initialize qubit to |1>"""
    circuit.x(qreg_data[0])
    circuit.barrier(qreg_data) # A barrier is a visual indicator of the grouping of a circuit section.
    return circuit

def encode_bit_flip(circuit, state, ancillas) -> QuantumCircuit: # -> is a return value annotation
    """Encode bit-flip. This is done by simply adding a cx"""
    for ancilla in ancillas:
        circuit.cx(state, ancilla)
    circuit.barrier(state, *ancillas)
    return circuit

def measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_measure):
    """
        Measure the syndrome by measuring the parity.
        We reset our ancilla qubits after measuring the stabilizer
        so we can reuse them for repeated stabilizer measurements.
        Because we have already observed the state of the qubit,
        we can write the conditional reset protocol directly to
        avoid another round of qubit measurement if we used
        the `reset` instruction.
    """
    circuit.cx(qreg_data[0], qreg_measure[0])
    circuit.cx(qreg_data[1], qreg_measure[0])
    circuit.cx(qreg_data[0], qreg_measure[1])
    circuit.cx(qreg_data[2], qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure) # * can pass arbitrary number argument
    circuit.measure(qreg_measure, creg_measure)
    # with circuit.if_test((creg_syndrome[0], 1)): # reset the ancilla bit
    #     circuit.x(qreg_measure[0])
    # with circuit.if_test((creg_syndrome[1], 1)):
    #     circuit.x(qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def apply_correction_bit(circuit, qreg_data, creg_syndrome):
    """We can detect where an error occurred and correct our state"""
    with circuit.if_test((creg_syndrome, 3)):
        circuit.x(qreg_data[0])
    with circuit.if_test((creg_syndrome, 1)):
        circuit.x(qreg_data[1])
    with circuit.if_test((creg_syndrome, 2)):
        circuit.x(qreg_data[2])
    circuit.barrier(qreg_data)
    return circuit

def apply_final_readout(circuit, qreg_data, creg_data):
    """Read out the final measurements"""
    circuit.barrier(qreg_data)
    circuit.measure(qreg_data, creg_data)
    return circuit

def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=True)
circuit.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
tcirc.draw("mpl")

In [ ]:
qreg_data = QuantumRegister(3)
qreg_measure = QuantumRegister(2)
creg_data = ClassicalRegister(3)
creg_syndrome = ClassicalRegister(2)
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]

def build_qc():
    """Build a typical error correction circuit"""
    return QuantumCircuit(qreg_data, qreg_measure, creg_data, creg_syndrome)

def initialize_qubits(circuit: QuantumCircuit):
    """Initialize qubit to |1>"""
    circuit.x(qreg_data[0])
    circuit.barrier(qreg_data) # A barrier is a visual indicator of the grouping of a circuit section.
    return circuit

def encode_bit_flip(circuit, state, ancillas) -> QuantumCircuit: # -> is a return value annotation
    """Encode bit-flip. This is done by simply adding a cx"""
    for ancilla in ancillas:
        circuit.cx(state, ancilla)
    circuit.barrier(state, *ancillas)
    return circuit

def measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_measure):
    """
        Measure the syndrome by measuring the parity.
        We reset our ancilla qubits after measuring the stabilizer
        so we can reuse them for repeated stabilizer measurements.
        Because we have already observed the state of the qubit,
        we can write the conditional reset protocol directly to
        avoid another round of qubit measurement if we used
        the `reset` instruction.
    """
    circuit.cx(qreg_data[0], qreg_measure[0])
    circuit.cx(qreg_data[1], qreg_measure[0])
    circuit.cx(qreg_data[0], qreg_measure[1])
    circuit.cx(qreg_data[2], qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure) # * can pass arbitrary number argument
    circuit.measure(qreg_measure, creg_measure)
    with circuit.if_test((creg_syndrome[0], 1)): # reset the ancilla bit
        circuit.x(qreg_measure[0])
    with circuit.if_test((creg_syndrome[1], 1)):
        circuit.x(qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def apply_correction_bit(circuit, qreg_data, creg_syndrome):
    """We can detect where an error occurred and correct our state"""
    with circuit.if_test((creg_syndrome, 3)):
        circuit.x(qreg_data[0])
    with circuit.if_test((creg_syndrome, 1)):
        circuit.x(qreg_data[1])
    with circuit.if_test((creg_syndrome, 2)):
        circuit.x(qreg_data[2])
    circuit.barrier(qreg_data)
    return circuit

def apply_final_readout(circuit, qreg_data, creg_data):
    """Read out the final measurements"""
    circuit.barrier(qreg_data)
    circuit.measure(qreg_data, creg_data)
    return circuit

def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=False)
circuit.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
tcirc.draw("mpl")

In [ ]:
qreg_data = QuantumRegister(3)
qreg_measure = QuantumRegister(2)
creg_data = ClassicalRegister(3)
creg_syndrome = ClassicalRegister(2)
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]

def build_qc():
    """Build a typical error correction circuit"""
    return QuantumCircuit(qreg_data, qreg_measure, creg_data, creg_syndrome)

def initialize_qubits(circuit: QuantumCircuit):
    """Initialize qubit to |1>"""
    circuit.x(qreg_data[0])
    circuit.barrier(qreg_data) # A barrier is a visual indicator of the grouping of a circuit section.
    return circuit

def encode_bit_flip(circuit, state, ancillas) -> QuantumCircuit: # -> is a return value annotation
    """Encode bit-flip. This is done by simply adding a cx"""
    for ancilla in ancillas:
        circuit.cx(state, ancilla)
    circuit.h(qreg_data[0])
    circuit.h(qreg_data[1])
    circuit.h(qreg_data[2])

    circuit.barrier(state, *ancillas)
    return circuit

def measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_measure):
    """
        Measure the syndrome by measuring the parity.
        We reset our ancilla qubits after measuring the stabilizer
        so we can reuse them for repeated stabilizer measurements.
        Because we have already observed the state of the qubit,
        we can write the conditional reset protocol directly to
        avoid another round of qubit measurement if we used
        the `reset` instruction.
    """
    circuit.cx(qreg_data[0], qreg_measure[0])
    circuit.cx(qreg_data[1], qreg_measure[0])
    circuit.cx(qreg_data[0], qreg_measure[1])
    circuit.cx(qreg_data[2], qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure) # * can pass arbitrary number argument
    circuit.measure(qreg_measure, creg_measure)
    with circuit.if_test((creg_syndrome[0], 1)): # reset the ancilla bit
        circuit.x(qreg_measure[0])
    with circuit.if_test((creg_syndrome[1], 1)):
        circuit.x(qreg_measure[1])
    circuit.barrier(*qreg_data, *qreg_measure)
    return circuit

def apply_correction_bit(circuit, qreg_data, creg_syndrome):
    """We can detect where an error occurred and correct our state"""
    with circuit.if_test((creg_syndrome, 3)):
        circuit.x(qreg_data[0])
    with circuit.if_test((creg_syndrome, 1)):
        circuit.x(qreg_data[1])
    with circuit.if_test((creg_syndrome, 2)):
        circuit.x(qreg_data[2])
    circuit.barrier(qreg_data)
    return circuit

def apply_final_readout(circuit, qreg_data, creg_data):
    """Read out the final measurements"""
    circuit.barrier(qreg_data)
    circuit.measure(qreg_data, creg_data)
    return circuit

def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=True)
circuit.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeVigo
from qiskit_aer import AerSimulator

# ideal
sim_ideal = AerSimulator()

# Execute and get counts
result = sim_ideal.run(transpile(circuit, sim_ideal)).result()
counts = result.get_counts(0)
plot_histogram(counts, title='Ideal counts for 3-qubit GHZ state')

In [ ]:
circ.draw("mpl")

In [ ]:
circuit.draw("mpl")

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

# Import from Qiskit Aer noise module
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError, pauli_error, depolarizing_error, thermal_relaxation_error)

import qiskit
from qiskit_aer import AerSimulator

###########

# Generate 3-qubit GHZ state
circ = qiskit.QuantumCircuit(3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure_all()

# Construct an ideal simulator
aersim = AerSimulator()

# Perform an ideal simulation
result_ideal = qiskit.execute(circ, aersim).result()
counts_ideal = result_ideal.get_counts(0)
print('Counts(ideal):', counts_ideal)
# Counts(ideal): {'000': 493, '111': 531}

# circ.draw(output="mpl", style='iqp')
circ.draw("mpl")

from qiskit import IBMQ, transpile
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

from qiskit.providers.fake_provider import FakeVigo
device_backend = FakeVigo()

# Construct quantum circuit
circ = QuantumCircuit(3, 3)
circ.h(0)
circ.cx(0, 1)
circ.cx(1, 2)
circ.measure([0, 1, 2], [0, 1, 2])

sim_ideal = AerSimulator()

# Execute and get counts
result = sim_ideal.run(transpile(circ, sim_ideal)).result()
counts = result.get_counts(0)
plot_histogram(counts, title='Ideal counts for 3-qubit GHZ state')

sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
# has correction
circ = QuantumCircuit(3, 1)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)

circ.barrier(0,1,2)
circ.x(0)

circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.ccx(1,2,0)
circ.barrier(0,1,2)
circ.measure([0], [0])
# circ.draw(output="mpl").savefig("Circuit Bit Flip Code (no syn).svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_sydney = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_sydney)

# Execute noisy simulation and get counts
result_noise = sim_sydney.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
# without decoding
circ = QuantumCircuit(3, 3)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
circ.barrier(0,1,2)
circ.measure([0,1,2], [0,1,2])
circ.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
# without encoding and decoding
circ = QuantumCircuit(1, 1)
circ.x(0)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
circ.barrier(0)
circ.measure([0], [0])
circ.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
# has correction
circ = QuantumCircuit(3, 1)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.barrier(0,1,2)
circ.z(0)
circ.barrier(0,1,2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.ccx(1,2,0)
circ.barrier(0,1,2)
circ.measure([0], [0])
# circ.draw(output="mpl").savefig("Circuit: Phase Flip Code (no syn).svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
# without encoding and decoding
import numpy as np

circ = QuantumCircuit(1, 1)
# circ.rx(np.pi/4,0)
circ.h(0)
circ.barrier(0)
circ.z(0)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
circ.barrier(0)
circ.h(0)
circ.measure([0], [0])
circ.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
# without encoding and decoding
import numpy as np

circ = QuantumCircuit(1, 1)
# circ.rx(np.pi/4,0)
circ.h(0)
# circ.barrier(0)
# circ.z(0)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.barrier(0,1,2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
circ.barrier(0)
circ.h(0)
circ.measure([0], [0])
circ.draw(output="mpl", style='iqp')

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Counts for 3-qubit GHZ state with device noise model")

In [ ]:
def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    # circuit = build_qc()
    circuit = QuantumCircuit(qreg_data, qreg_measure, creg_data)
    # circuit = initialize_qubits(circuit)
    circuit.h(qreg_data[0])
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    # circuit.x(qreg_data[0])
    # circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    # if apply_correction:
    #     circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=True)
# circuit.draw(output="mpl", style='iqp')
circuit.draw(output="mpl").savefig("Circuit: Bit Flip Code_encoding.svg")
circuit.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()
# Transpile the circuit for the noisy basis gates
tcirc = transpile(circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
# plot_histogram(counts_noise, title=r"Bit Flip Code: $(|0_L\rangle + |1_L\rangle)\ /\ \sqrt {2}$").savefig("Bit Flip Code_encoding.svg")
plot_histogram(counts_noise, title=r"Bit Flip Code: $(|0_L\rangle + |1_L\rangle)\ /\ \sqrt {2}$")

In [ ]:
def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit.x(qreg_data[0])
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)
    circuit.barrier(*qreg_data, *qreg_measure)
    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=False)
# circuit.draw(output="mpl", style='iqp')
circuit.draw(output="mpl").savefig("Circuit: Bit Flip Code_Correctable Fault no Correction.svg")
circuit.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Bit Flip Code: Correctable Fault w/o Correction").savefig("Bit Flip Code_Correctable Fault no Correction.svg", bbox_inches='tight')
plot_histogram(counts_noise, title="Bit Flip Code: Correctable Fault w/o Correction")

In [ ]:
def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit.x(qreg_data[0])
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=True)
# circuit.draw(output="mpl", style='iqp')
circuit.draw(output="mpl").savefig("Circuit: Bit Flip Code_Correctable Fault w Correction.svg")
circuit.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, figsize=(7, 4.5), title="Bit Flip Code: Correctable Fault w/ Correction").savefig("Bit Flip Correctable Fault w Correction.svg", bbox_inches='tight')
plot_histogram(counts_noise,figsize=(7, 4.5), title="Bit Flip Code: Correctable Fault w/ Correction")

In [ ]:
def build_error_correction_sequence(apply_correction: bool) -> QuantumCircuit:

    circuit = build_qc()
    circuit = initialize_qubits(circuit)
    circuit = encode_bit_flip(circuit, state_data, ancillas_data)
    circuit.x(qreg_data[0])
    circuit.x(qreg_data[1])
    circuit = measure_syndrome_bit(circuit, qreg_data, qreg_measure, creg_syndrome)

    if apply_correction:
        circuit = apply_correction_bit(circuit, qreg_data, creg_syndrome)

    circuit = apply_final_readout(circuit,  qreg_data, creg_data)
    return circuit

circuit = build_error_correction_sequence(apply_correction=True)
# circuit.draw(output="mpl", style='iqp')
circuit.draw(output="mpl").savefig("Circuit: Bit Flip Code_Non-correctable Fault w Correction.svg")
circuit.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circuit, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Bit Flip Code: Non-correctable Fault w/ Correction").savefig("Bit Flip Non-correctable Fault w Correction.svg", bbox_inches='tight')
plot_histogram(counts_noise, title="Bit Flip Code: Non-correctable Fault w/ Correction")


In [ ]:
# has correction
circ = QuantumCircuit(3, 3)
# circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.barrier(0,1,2)
# circ.z(0)
# circ.barrier(0,1,2)
# circ.h(0)
# circ.h(1)
# circ.h(2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
# circ.barrier(0,1,2)
circ.measure([0,1,2], [0,1,2])
circ.draw(output="mpl").savefig("Circuit: Phase Flip Code (no syn)_encoding_0.svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
# plot_histogram(counts_noise, title=r"Phase Flip Code: $|0_L\rangle$").savefig("Phase Flip Code_encoding_0.svg")
plot_histogram(counts_noise, title=r"Phase Flip Code: $|0_L\rangle$")

In [ ]:
# has correction
circ = QuantumCircuit(3, 3)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.barrier(0,1,2)
# circ.z(0)
# circ.barrier(0,1,2)
# circ.h(0)
# circ.h(1)
# circ.h(2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
# circ.barrier(0,1,2)
circ.measure([0,1,2], [0,1,2])
circ.draw(output="mpl").savefig("Circuit: Phase Flip Code (no syn)_encoding_1.svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title=r"Phase Flip Code: $|1_L\rangle$").savefig("Phase Flip Code_encoding_1.svg")
plot_histogram(counts_noise, title=r"Phase Flip Code: $|1_L\rangle$")

In [ ]:
# has correction
circ = QuantumCircuit(3, 1)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.barrier(0,1,2)
circ.z(0)
circ.barrier(0,1,2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.ccx(1,2,0)
circ.barrier(0,1,2)
circ.measure([0], [0])
circ.draw(output="mpl").savefig("Circuit: Phase Flip Code (no syn)_Correctable Fault w Correction.svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Phase Flip Code: Correctable Fault w/ Correction").savefig("Phase Flip Code: Correctable Fault w Correction.svg")
plot_histogram(counts_noise, title="Phase Flip Code: Correctable Fault w/ Correction")

In [ ]:
# has correction
circ = QuantumCircuit(3, 1)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.barrier(0,1,2)
circ.z(0)
circ.barrier(0,1,2)
circ.h(0)
circ.h(1)
circ.h(2)
# circ.cx(0, 1)
# circ.cx(0, 2)
# circ.ccx(1,2,0)
circ.barrier(0,1,2)
circ.measure([0], [0])
circ.draw(output="mpl").savefig("Circuit: Phase Flip Code (no syn)_Correctable Fault no Correction.svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Phase Flip Code: Correctable Fault w/o Correction").savefig("Phase Flip Code: Correctable Fault no Correction.svg")
plot_histogram(counts_noise, title="Phase Flip Code: Correctable Fault w/o Correction")

In [ ]:
# has correction
circ = QuantumCircuit(3, 1)
circ.x(0)
circ.barrier(0,1,2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.barrier(0,1,2)
circ.z(0)
circ.z(1)
circ.barrier(0,1,2)
circ.h(0)
circ.h(1)
circ.h(2)
circ.cx(0, 1)
circ.cx(0, 2)
circ.ccx(1,2,0)
circ.barrier(0,1,2)
circ.measure([0], [0])
circ.draw(output="mpl").savefig("Circuit: Phase Flip Code (no syn)_Non-correctable Fault w Correction.svg")
circ.draw(output="mpl")

In [ ]:
from qiskit.providers.fake_provider import FakeSydney
from qiskit_aer import AerSimulator

device_backend = FakeSydney()
sim_vigo = AerSimulator.from_backend(device_backend)
# sim_vigo = AerSimulator()

# Transpile the circuit for the noisy basis gates
tcirc = transpile(circ, sim_vigo)

# Execute noisy simulation and get counts
result_noise = sim_vigo.run(tcirc).result()
counts_noise = result_noise.get_counts(0)
plot_histogram(counts_noise, title="Phase Flip Code: Non-correctable Fault w/ Correction").savefig("Phase Flip Code: Non-correctable Fault w Correction.svg")
plot_histogram(counts_noise, title="Phase Flip Code: Non-correctable Fault w/ Correction")